In [ ]:
import re
import time
import glob
import pywt

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV


import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

# import ipywidgets as widgets
# from IPython.display import display

# %matplotlib widget
%matplotlib inline

### Concatenate all the files

In [ ]:
# get path to all files
path = '../spectra/simulated_data/'
files = sorted(glob.glob(path + 'model_parameters_data*.txt'), key=lambda x: int(re.search(r'\d+', x).group()))
# write_data = open('model_parameters_data_all.txt', 'w')
# write_params  = open('model_parameters_all.txt', 'w')

# # start timer
# start_timer = time.time()
# # read in all files
# for file in files:
#     with open(file, 'r') as f:
#         # skip the first 10 lines in each file
#         for i in range(10):
#             f.readline()
#             # read in the data from each file
#         data = f.readlines()
#         # get the length of the data in each file
#         length = len(data)
#         # write the data to the output file in the same order as the files
#         for i in range(length):
#             write_data.write(data[i])
            
# # close the files
# write_data.close()
# # # check how much time it took
# print(time.time() - start_timer)      

In [ ]:
# start_timer = time.time()
# # get the params data
# df_param = pd.DataFrame()
# for file in files:
#     # get the parameters used
#     df_param = df_param.append(pd.read_csv(file, skiprows=1, nrows=6, header=None, sep=' ', names=['A', 'B', 'C', 'D']))
#     # save the dataframe to a csv file
#     df_param.to_csv('model_parameters_data.csv', index=False)
    
# print(time.time() - start_timer)

#### Load the files 

In [ ]:
start_time = time.time()
df_data = pd.read_csv('model_parameters_data_all.txt', sep=' ', header=None) # all data dataframe
print(time.time() - start_time)
df_data.head()

In [ ]:
df_data.shape

In [ ]:
df_param = pd.read_csv('model_parameters_data.csv')
df_param.head()

In [ ]:
# drop the second and fourth column
df_param.drop(['B', 'D'], axis=1, inplace=True)

In [ ]:
# transform the data to the right dataframe
df_param = df_param.assign(g = df_param.groupby('A').cumcount()).pivot(index='g', columns='A', values='C')

In [ ]:
df_param.head(10)

In [ ]:
df_param.columns

In [ ]:
columns = ['Frequency', 'Intensity']
df_data.columns = columns
df_data.shape

In [ ]:
df_data.head()

In [ ]:
freq_sig = np.array(np.array_split(df_data['Frequency'], len(files)))
signal = np.array(np.array_split(df_data['Intensity'], len(files)))

# plot the signal in one plot
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")
plt.plot(freq_sig[0],  signal[0], color='r', label='Spectrum 1')
plt.plot(freq_sig[1], signal[1], color='g', label='Spectrum 2')
plt.plot(freq_sig[2], signal[2], color='b', label='Spectrum 3')
plt.plot(freq_sig[3], signal[3], color='y', label='Spectrum 4')
plt.plot(freq_sig[4], signal[4], color='k', label='Spectrum 5')
plt.xlabel("Frequency (GHz)")
plt.ylabel("Intensity (K)")
plt.legend()
plt.show()

In [ ]:
# plot the data in separate plots
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(15, 6))
sns.set_style("whitegrid")
for i, ax in enumerate(axes.ravel(), start=1):
    ax.plot(freq_sig[i-1], signal[i-1], label='Spectrum {}'.format(i))
    ax.set_xlabel("Frequency (GHz)", fontdict={'fontsize': 14})
    ax.set_ylabel("Intensity (K)", fontdict={'fontsize': 14})
    ax.set_title("Spectrum {}".format(i))
    
# plt.savefig('../spectra/simulated_data/spectrum_plots.png')

In [ ]:
# print(pywt.wavelist())

### Feature Extraction - Wavelet Decomposition

In [ ]:
#create a  feature vector array for each spectrum 
feature_vector = np.zeros((len(signal), int(df_data.shape[0]/len(files))))
detail_coeffs = np.zeros((len(signal), 558))
approx_coeffs = np.zeros((len(signal), 558))
level = 6
wname = 'db35'
def decompose_signal_dwt(_signal, wavelet=wname, mode='per', level=level):
    """
    Performs wavelet denoising on the given signal.
    """
    # loop throuh all the spectra 
    for spectra_index in range(len(_signal)):
        # max_level = pywt.dwt_max_level(len(_signal[spectra_index]), wavelet)
        coeffs = pywt.wavedec(_signal[spectra_index], wavelet=wavelet, mode=mode, level=level)
        coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
        detail_coeffs[spectra_index, :] = coeff_arr[coeff_slices[1]['d']] # 4th level detail coeffs 
        # get the approximation coeffs
        approx_coeffs[spectra_index, :] = coeffs[0] # 4th level approximation coeffs
        
        reconstructed_signal = pywt.waverec(coeffs, wavelet=wavelet, mode=mode)
        feature_vector[spectra_index, :] = coeff_arr[:int(df_data.shape[0]/len(files))]
        # add the coeff_arr to the dataframe for each spectra
        # df_data['fv_dwt_{}'.format(wavelet)] = pd.Series(feature_vector.reshape(1, -1)[0], index=df_data.index) #TODO: find a way to optimize (or comment it out)
        
    return coeff_arr, coeff_slices

In [ ]:
feature_vector.shape

In [ ]:
start_timer = time.time()
coeff_arr, coeff_slices = decompose_signal_dwt(signal)
print(time.time() - start_timer)
df_data

In [ ]:
detail_coeffs.shape
detail_coeffs

## Wavelet Decomposition Plots

In [ ]:
def decomposed_dwt_detail_coeffs_plots(_signal, wavelet=wname, level=level):
    """
    Plots of the detail coeffs of the signal.
    """
    # plot the reconstructed signal and the original signal in one plot
    for spectra_index in range(len(_signal)):                     
        # compute the maximum useful level of decomposition for each wavelet                        
        # max_level = pywt.dwt_max_level(len(_signal[spectra_index]), wavelet)
        fig, axes = plt.subplots(nrows=1, ncols=level, dpi=400, sharey='none', sharex='all', figsize=(18, 5))
        sns.set_style("whitegrid")
        for i, ax in enumerate(axes.ravel(), start=1):
            ax.plot(coeff_arr[coeff_slices[i]['d']], label='Level {}'.format(i))
            ax.set_xlabel("Frequency (GHz)", fontdict={'fontsize': 14})
            ax.set_ylabel("Intensity (K)", fontdict={'fontsize': 14})
            ax.set_title("Spectrum {} detail coeffiecients at level {} for {} ".format(spectra_index+1, i, wavelet)) 
            
        plt.show()

In [ ]:
# decomposed_dwt_detail_coeffs_plots(signal) 

In [ ]:
# TODO: find a way to plot the detail coefficients of the signal
def decomposed_dwt_approx_coeffs_plots(_signal, wavelet=wname, level=level):
    
    for spectra_index in range(len(_signal)):
        fig, axes = plt.subplots(nrows=1, ncols=level, figsize=(10, 6))
        sns.set_style("whitegrid")
        for i, ax in enumerate(axes.ravel(), start=1):
            ax.plot(coeff_arr[coeff_slices[1]['d']])
            ax.set_title("Spectrum {} approximation coeffiecients at level {} for {} ".format(spectra_index+1, i,  wavelet))
            
        plt.show()

In [ ]:
# decomposed_dwt_approx_coeffs_plots(signal)

### Get Features and Labels

In [ ]:
# fv = np.zeros((len(signal), int(df.shape[0]/len(files))))
# for spectra_index in range(len(signal)):
#     # get the level 3 detail coefficients
#     detail_coeffs = coeff_arr[coeff_slices[3]['d']]

# fv  = detail_coeffs
# labels =dff

In [ ]:
# len(signal)

approx_coeffs.shape

 #### Have a glimpse look at any of the signal and its generated detail and approximation coefficients 

In [ ]:

# for spectra_index in range(len(signal)):                     
#         # compute the maximum useful level of decomposition for each wavelet                        
#         # max_level = pywt.dwt_max_level(len(_signal[spectra_index]), wavelet)
#         fig, axes = plt.subplots(nrows=1, ncols=10, sharey='none', sharex='all', figsize=(20, 7))
#         sns.set_style("whitegrid")
#         for i, ax in enumerate(axes.ravel(), start=1):
#             ax.plot(signal[spectra_index], label='Spectrum {}'.format(spectra_index))
#             # ax.set_xlabel("Frequency (GHz)", fontdict={'fontsize': 14})
#             # ax.set_ylabel("Intensity (K)", fontdict={'fontsize': 14})
#             # ax.set_title("Spectrum {} detail coeffiecients at level {} for {} ".format(spectra_index+1, i, wavelet)) 

#     # ax[0,1].plot(signal[spectra_index], label='original')
#     # ax[1,1].plot(approx_coeffs[spectra_index], label='spectrum {} approx coeff'.format(spectra_index+1))
#     # ax[2,1].plot(detail_coeffs[spectra_index], label='spectrum {} detail coeff'.format(spectra_index+1))
# plt.legend()
# plt.show()

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(signal[243], label='original - 243')
plt.legend()
plt.savefig("5K_data/original.png")
plt.show()

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(approx_coeffs[243], label='approx coeff - 243')
plt.legend()
plt.savefig("5K_data/approx_level6.png")
plt.show()

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(detail_coeffs[243], label='detail - 243')
plt.legend()
plt.savefig("5K_data/detail_level6.png")
plt.show()

In [ ]:
features = detail_coeffs
labels_ = df_param

print('feature_shape: ', features.shape, 'labels_shape: ', labels_.shape)


In [ ]:
# drop the tcmb column
labels_.drop(columns=['tcmb'], inplace=True)

In [ ]:
labels_.head()

#### save the true params to a file

In [ ]:
# get the last 1500 data of the parameters from labels 
df_param = df_param.iloc[-1500:, :]
# save the vals to a csv file
df_param.to_csv("5K_data/true_param_vals.csv")
df_param.head()


In [ ]:
labels_

In [ ]:
df = pd.DataFrame(approx_coeffs)
# df = pd.concat([approx_coeffs, labels_], axis=1)

In [ ]:
df

In [ ]:
# convert the ntot column to log values 
labels_['ntot'] = np.log(labels_['ntot'])

In [ ]:
labels_

In [ ]:
df = pd.concat([df, labels_], axis=1)

In [ ]:
df

In [ ]:
X = df.iloc[:, :558]
y = df.iloc[:, -5:]


In [ ]:
X

In [ ]:
y

#### split the data into the training and test set

In [ ]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    shuffle=False,
    random_state=42
    )

print('X_train shape: ',  X_train.shape, '\n',
    'y_train shape: ', y_train.shape, '\n',
    'X_test shape: ', X_test.shape, '\n',
    'y_test shape: ', y_test.shape)

## 1. Multioutput Regressor - RF

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler



# regr_multirf = MultiOutputRegressor(
#     RandomForestRegressor(
#         max_depth=50
#         ))

# tuned model to use instantly
regr_multirf = MultiOutputRegressor(
    estimator=RandomForestRegressor(
        bootstrap='False',
        max_depth=90,
        max_features='log2',
        max_samples=0.8999999999999999,
        n_estimators=700,
        n_jobs=-1,
        random_state=42
))


# pipe = make_pipeline(StandardScaler(), regr_multirf)
# pipe.fit(X_train, y_train)  # apply scaling on training data


# pipe.score(X_test, y_test)  # apply scaling on testing data, without leaking training data.

# multioutput regressor
regr_multirf.fit(X_train, y_train)

In [ ]:
#  predict on the new test data
y_multirf_pred = regr_multirf.predict(X_test)

In [ ]:
regr_multirf.score(X_test, y_test)

### save the predicted parameters to a csv file 

In [ ]:
param_df_rf = pd.DataFrame(y_multirf_pred, columns=['fwhm_rf_pred', 'ntot_rf_pred', 'size_rf_pred', 'tex_rf_pred', 'vlsr_rf_pred'])
param_df_rf.head()

In [ ]:
# convert back the log to linear values 
param_df_rf['ntot_rf_pred'] = np.exp(param_df_rf['ntot_rf_pred'])

In [ ]:
param_df_rf.to_csv('5K_data/predicted_parameters_rf.csv')
param_df_rf.head()

In [ ]:
pred_para = pd.read_csv('5K_data/predicted_parameters_rf.csv')
true_para = pd.read_csv('5K_data/true_param_vals.csv')

# add the predicted value to the true value dataframe as new columns separated by an empty column 
true_para = pd.concat([true_para,  pd.DataFrame(np.zeros(len(true_para))), param_df_rf], axis=1)
true_para.to_csv('5K_data/true_param_vals_with_predicted_rf.csv')
true_para.head()

In [ ]:
# print('true val: {}'.format(y_test.iloc[:, 0]), 'pred_value: {}'.format( y_multirf_pred[:,0]))

In [ ]:
# print('true val: {}'.format(y_test.iloc[:, 1]), 'pred_value: {}'.format( y_multirf_pred[:,1]))

In [ ]:
# print('true val: {}'.format(y_test.iloc[:, 2]), 'pred_value: {}'.format( y_multirf_pred[:,2]))

In [ ]:
# print('true val: {}'.format(y_test.iloc[:, 3]), 'pred_value: {}'.format( y_multirf_pred[:,3]))

In [ ]:
# print('true val: {}'.format(y_test.iloc[:, 4]), 'pred_value: {}'.format( y_multirf_pred[:,4]))

#### Metrics

In [ ]:
class RegressionMetrics:
    def __init__(self):
        self.metrics = {
            "mae": self._mean_absolute_error,
            "mse": self._mean_squared_error,
            "rmse": self._root_mean_squared_error,
            "mape": self._mean_absolute_percentage_error,
            "r2": self._r2_score,
            "msle": self._mean_squared_log_error,
            # "rmsle": self._root_mean_squared_logarithmic_error,
        }

    def get_metric(self, metric,  y_true, y_pred):
        if metric not in self.metrics:
            raise Exception("Metric not found")
        
        if metric == "mae":
            return self._mean_absolute_error(y_true, y_pred)
        if metric == "mse":
            return self._mean_squared_error(y_true, y_pred)
        if metric == "rmse":
            return self._root_mean_squared_error(y_true, y_pred)
        if metric == "mape":
            return self._mean_absolute_percentage_error(y_true, y_pred)
        if metric == "r2":
            return self._r2_score(y_true, y_pred)
        if metric == "msle":
            return self._mean_squared_log_error(y_true, y_pred)
        # if metric == "rmsle":
        #     return self._root_mean_squared_logarithmic_error(y_true, y_pred)
        
    @staticmethod
    def _mean_absolute_error(y_true, y_pred):
        return metrics.mean_absolute_error(y_true, y_pred)

    @staticmethod
    def _mean_squared_error(y_true, y_pred):
        return metrics.mean_squared_error(y_true, y_pred)

    def _root_mean_squared_error(self, y_true, y_pred):
        return np.sqrt(metrics.mean_squared_error(y_true, y_pred))

    def _mean_absolute_percentage_error(self, y_true, y_pred):
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    @staticmethod
    def _r2_score(y_true, y_pred):
        return metrics.r2_score(y_true, y_pred)
    
    def _mean_squared_log_error(self, y_true, y_pred):
        return np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    
    # TODO: investigate in the case where it gives an error
    # def _root_mean_squared_logarithmic_error(self, y_true, y_pred):
    #     return np.sqrt(np.mean(np.square(np.log(y_pred + 1) - np.log(y_true + 1))))


In [ ]:
# get the metricl for the multirf regressor
metrics_multirf = RegressionMetrics()
_metrics = ['mae', 'mse', 'rmse', 'mape', 'r2', 'msle']
for metric in _metrics:
    print("Multirf  {}: ".format(metric), metrics_multirf.get_metric(metric, y_test, y_multirf_pred))

### Metrics on individual predicted parameters - Random Forest

In [ ]:
print("fwhm MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,0], y_multirf_pred[:,0]))
print("ntot MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,1], y_multirf_pred[:,1]))
print("size MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,2], y_multirf_pred[:,2]))
print("tex MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,3], y_multirf_pred[:,3]))
print("vlsr MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,4], y_multirf_pred[:,4]))

print('\n')
print("fwhm MAE:%.4f" % metrics.mean_absolute_error(y_test.iloc[:,0], y_multirf_pred[:,0]))
print("ntot MAE:%.4f" % metrics.mean_absolute_error(y_test.iloc[:,1], y_multirf_pred[:,1]))
print("size MAE:%.4f" % metrics.mean_absolute_error(y_test.iloc[:,2], y_multirf_pred[:,2]))
print("tex MAE:%.4f" % metrics.mean_absolute_error(y_test.iloc[:,3], y_multirf_pred[:,3]))
print("vlsr MAE:%.4f" % metrics.mean_absolute_error(y_test.iloc[:,4], y_multirf_pred[:,4]))

print('\n')
print("fwhm MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,0], y_multirf_pred[:,0]))
print("ntot MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,1], y_multirf_pred[:,1]))
print("size MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,2], y_multirf_pred[:,2]))
print("tex MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,3], y_multirf_pred[:,3]))
# print("tex MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,4], y_multirf_pred[:,4])) # TODO: normalize the values to have only +v values in the test-train set


print('\n')
# print("fwhm MAPE:%.4f" % np.mean(np.abs((y_test.iloc[:,0] - y_multirf_pred[:,0]) / y_test.iloc[:,0])) * 100)
# print("ntot MAPE:%.4f" % np.mean(np.abs((y_test.iloc[:,1] - y_multirf_pred[:,1]) / y_test.iloc[:,1])) * 100)
# print("size MAPE:%.4f" %np.mean(np.abs((y_test.iloc[:,2] - y_multirf_pred[:,2]) / y_test.iloc[:,2])) * 100)
# print("tex MAPE:%.4f" % np.mean(np.abs((y_test.iloc[:,3] - y_multirf_pred[:,3]) / y_test.iloc[:,3])) * 100)
# print("vlsr MAPE:%.4f" % np.mean(np.abs((y_test.iloc[:,4] - y_multirf_pred[:,4]) / y_test.iloc[:,4])) * 100)

## Predicted vs True values plots

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(45,35), dpi=150)
plt.setp(ax.get_xticklabels(), fontsize=14)
sns.set_theme(font_scale=2) 
axes[2,1].set_visible(False)
axes[2,0].set_position([0.30, 0.1,0.40,0.25])

g1 = sns.regplot(x=y_test.iloc[:,0], y=y_multirf_pred[:,0], color='blue', ax=axes[0,0]) # fwhm
g2 = sns.regplot(x=y_test.iloc[:,1], y=y_multirf_pred[:,1], color='blue', ax=axes[0,1]) # column density
g3 = sns.regplot(x=y_test.iloc[:,2], y=y_multirf_pred[:,2], color='blue', ax=axes[1,0]) # size
g4 = sns.regplot(x=y_test.iloc[:,3], y=y_multirf_pred[:,3], color='blue', ax=axes[1,1]) # tex
g5 = sns.regplot(x=y_test.iloc[:,4], y=y_multirf_pred[:,4], color='blue', ax=axes[2,0]) # vlsr

g1.set(title='FWHM', ylabel="predicted values", xlabel="true values")
g2.set(title='Column density', ylabel="predicted values", xlabel="true values", xscale='log', yscale='log')
g3.set(title='Size', ylabel="predicted values", xlabel="true values")
g4.set(title='Tex', ylabel="predicted values", xlabel="true values")
g5.set(title='Vlsr', ylabel="predicted values", xlabel="true values")

plt.savefig("5K_data/pred_true_5K_RF.png")
plt.show()

### 3D plots -  Columnn density, Excitation temperature and Size

In [ ]:
pred_param_rf = pd.DataFrame(y_multirf_pred, columns=['fwhm', 'ntot', 'size', 'tex', 'vlsr'])
pred_param_rf.head()

### Residuals 3D plot -  Random Forest Regressor

In [ ]:
fig = plt.figure(figsize=(26, 11), dpi=120)
plt.setp(ax.get_xticklabels(), fontsize=12)
ax1 = fig.add_subplot(121, projection='3d')

markers = ['D', 's', '.']
labels = ['ntot', 'tex', 'size']
colors = ['black', 'red', 'blue']

residuals_rf = (y_test - y_multirf_pred)
# residuals
x1 = residuals_rf['ntot']
y1 = residuals_rf['tex']
z1 = residuals_rf['size']

for i in range(len(markers)):
    ax1.scatter3D(x1, y1, z1, marker=markers[i], c=colors[i], label=labels[i])

ax1.set_title('ntot-tex-size distribution - Residuals')
ax1.set_xlabel('Column Density')
ax1.set_ylabel('Excitation Temperature')
ax1.set_zlabel('Size')
plt.legend(loc="best")
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(45,35), dpi=250)
plt.setp(ax.get_xticklabels(), fontsize=14)
sns.set_theme(font_scale=2) 
axes[2,1].set_visible(False)
axes[2,0].set_position([0.30, 0.1,0.40,0.25])

# plot the redisual distribution using seaborn
g1 = sns.residplot(x=y_multirf_pred[:,0], y=residuals_rf['fwhm'], lowess=True, ax=axes[0,0]) # fwhm
g2 = sns.residplot(x=y_multirf_pred[:,1], y=residuals_rf['ntot'], lowess=True, ax=axes[0,1]) # column density
g3 = sns.residplot(x=y_multirf_pred[:,2], y=residuals_rf['size'], lowess=True, ax=axes[1,0]) # size
g4 = sns.residplot(x=y_multirf_pred[:,3], y=residuals_rf['size'], lowess=True, ax=axes[1,1]) # tex
g5 = sns.residplot(x=y_multirf_pred[:,4], y=residuals_rf['vlsr'], lowess=True, ax=axes[2,0]) # vlsr

g1.set(title='FWHM', ylabel="residuals", xlabel="predicted values")
g2.set(title='Column density', ylabel="residuals", xlabel="predicted values")
g3.set(title='Size', ylabel="predicted values", xlabel="predicted values")
g4.set(title='Tex', ylabel="residuals", xlabel="predicted values")
g5.set(title='Vlsr', ylabel="residuals", xlabel="predicted values")

plt.savefig("5K_data/residuals_RF_5K.png")
plt.show()

In [ ]:
fig = plt.figure(figsize=(35, 25), dpi=400)
plt.setp(ax.get_xticklabels(), fontsize=14)
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

markers = ['D', 's', '.']
labels = ['ntot', 'tex', 'size']
colors = ['black', 'red', 'blue']

x2 = y_test['ntot']
y2 = y_test['tex']
z2 = y_test['size']


for i in range(len(markers)):
    x1 = pred_param_rf['ntot']
    y1 = pred_param_rf['tex']
    z1 = pred_param_rf['size']
    
    ax1.scatter3D(x1, y1, z1, marker=markers[i], c=colors[i], label=labels[i])
    ax2.scatter3D(x2, y2, z2, marker=markers[i], color=colors[i], label=labels[i])

ax1.set_title('ntot-tex-size distribution - Predicted Values')
ax1.set_xlabel('Column Density')
ax1.set_ylabel('Excitation Temperature')
ax1.set_zlabel('Size')


ax2.set_title('ntot-tex-size distribution - True Values')
ax2.set_xlabel('Column Density')
ax2.set_ylabel('Excitation Temperature')
ax2.set_zlabel('Size')
plt.legend(loc="best")

plt.savefig("5K_data/scatter3D_pred_true_5K_RF.png")
plt.show()

All the points are taking up the same position in a 3D space. Not sure if this is how its supposed to be. From my understanding, the Column density is dependent on Temperature, so they take the `x`, and `y` positions while size takes the `z` position.plt

## Hyperparameter Tuning for Random Forest

In [ ]:
multirf_model = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        criterion="squared_error",
        bootstrap=True,
        n_jobs=-1,
        max_samples=None,
))

multrf_hyperparameters = dict(
    estimator__n_estimators=np.arange(100, 1000, 100),
    estimator__max_depth=np.arange(10, 150, 10),
    estimator__min_samples_split=np.arange(2, 10, 2),
    estimator__min_samples_leaf=np.arange(1, 5, 1),
    # estimator__min_weight_fraction_leaf=np.arange(0, 0.5, 0.1),
    # estimator__criterion=["squared_error", "absolute_error"],
    estimator__max_features=["auto", "sqrt", "log2"],
    # estimator__max_samples=np.arange(0.5, 1, 0.1),
    estimator__bootstrap=["True", "False"]
    )




In [ ]:
random_search = RandomizedSearchCV(
    estimator=multirf_model,
    param_distributions=multrf_hyperparameters,
    n_iter=100,
    cv=5,
    verbose=2,
    error_score="raise",
    n_jobs=-1,
    random_state=42,
    return_train_score=True
)

In [ ]:
hyper_rf_tuned_model = random_search.fit(X_train, y_train)

In [ ]:
print("Best hyperparameters: ", hyper_rf_tuned_model.best_params_)
print("Best score: ", hyper_rf_tuned_model.best_score_)

In [ ]:
tuned_rf_model = hyper_rf_tuned_model.best_estimator_
y_multrf_tuned_tf = tuned_rf_model.predict(X_test)

In [ ]:
# # TODO: interpret the model evaluation metrics
# eval = RegressionMetrics()
# for metric in _metrics:
#     print(metric, ":", eval.get_metric(metric, y_test, y_multrf_tuned_tf))

## 2. XGBoost Regressor

In [ ]:
multixgb_model = MultiOutputRegressor(
    xgb.XGBRegressor(
        n_estimators=100,
        max_depth=10,
        max_leaves=10,
        max_bin=10,
        learning_rate=0.1,
        n_jobs=-1,
        gamma=0,
        min_child_weight=1.0,
        max_delta_step=0,
        importance_type="gain",
        eval_metric=metrics.mean_squared_error
    )
)

# xgb_hyperparameters = dict(
#     estimator__n_estimators=np.arange(100, 1000, 100),
#     estimator__max_depth=np.arange(10, 150, 10),
#     estimator__max_leaves=np.arange(10, 150, 10),
#     estimator__max_bin=np.arange(10, 150, 10),
#     # estimator_growth_policy=0,
#     estimator__learning_rate=np.arange(0.1, 1, 0.1),
#     estimator__min_child_weight=np.arange(1.0, 5.0, 0.5),
#     estimator__max_delta_step=np.arange(0, 10, 1),
#     estimator__importance_type=["gain", "weight", "cover", "total_gain", "total_cover"],
#     estimator__eval_metric=[metrics.mean_absolute_error, metrics.mean_squared_error]
# )

# xgbr_rand_search = RandomizedSearchCV(
#     estimator=multixgb_model,
#     param_distributions=xgb_hyperparameters,
#     n_iter=100,
#     cv=3
# )

# xgb_hyperparameters_tuning = xgbr_rand_search.fit(X_train, y_train)
# print('Best Parameters = {}'.format(xgb_hyperparameters_tuning.best_params_))

# xgb_tuned_model = xgb_hyperparameters_tuning.best_estimator_

multixgb_model.fit(X_train, y_train)

In [ ]:
y_multixgb_pred = multixgb_model.predict(X_test)

In [ ]:
for metric in _metrics:
    print(metric, ":", metrics_multirf.get_metric(metric, y_test, y_multixgb_pred))

### Metrics on individual predicted parameters - XGB Regressor

In [ ]:
print("fwhm MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,0], y_multixgb_pred[:,0]))
print("ntot MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,1], y_multixgb_pred[:,1]))
print("size MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,2], y_multixgb_pred[:,2]))
print("tex MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,3], y_multixgb_pred[:,3]))
print("vlsr MSE:%.4f" % metrics.mean_squared_error(y_test.iloc[:,4], y_multixgb_pred[:,4]))

print('\n')
print("fwhm MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,0], y_multixgb_pred[:,0]))
print("ntot MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,1], y_multixgb_pred[:,1]))
print("size MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,2], y_multixgb_pred[:,2]))
# print("tex MSLE:%.4f" % metrics.mean_squared_log_error(y_test.iloc[:,3], y_multixgb_pred[:,3]))

### save the predicted parameters to a csv file

In [ ]:
param_df_xgb = pd.DataFrame(y_multixgb_pred, columns=['fwhm_pred_xgb', 'ntot_pred_xgb', 'size_pred_xgb', 'tex_pred_xgb', 'vlsr_xgb_pred'])
param_df_xgb['ntot_pred_xgb'] = np.exp(param_df_xgb['ntot_pred_xgb'])
param_df_xgb.to_csv('5K_data/predicted_parameters_xgb.csv')

In [ ]:
param_df_xgb.head()

### Predicted vs True values - XGB Regressor

In [ ]:
X_train

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(45,35), dpi=150)
plt.setp(ax.get_xticklabels(), fontsize=14)
sns.set_theme(font_scale=2) 
axes[2,1].set_visible(False)
axes[2,0].set_position([0.30, 0.1,0.40,0.25])

g1 = sns.regplot(x=y_test.iloc[:, 0], y=y_multixgb_pred[:,0], color='blue', ax=axes[0,0]) # fwhm
g2 = sns.regplot(x=y_test.iloc[:,1], y=y_multixgb_pred[:,1], color='blue', ax=axes[0,1]) # column density
g3 = sns.regplot(x=y_test.iloc[:,2], y=y_multixgb_pred[:,2], color='blue', ax=axes[1,0]) # size
g4 = sns.regplot(x=y_test.iloc[:,3], y=y_multixgb_pred[:,3], color='blue', ax=axes[1,1]) # tex
g5 = sns.regplot(x=y_test.iloc[:,4], y=y_multixgb_pred[:,4], color='blue', ax=axes[2,0]) # vlsr

g1.set(title='FWHM', ylabel="predicted values", xlabel="true values")
g2.set(title='Column density', ylabel="predicted values", xlabel="true values", yscale="log", xscale="log")
g3.set(title='Size', ylabel="predicted values", xlabel="true values")
g4.set(title='Tex', ylabel="predicted values", xlabel="true values")
g5.set(title='Vlsr', ylabel="predicted values", xlabel="true values")

plt.savefig("5K_data/pred_true_5K_XGB.png")
plt.show()

### 3D plots - Columnn density, Excitation temperature and Size - XGB Regressor

In [ ]:
pred_param_xgb = pd.DataFrame(y_multixgb_pred, columns=['fwhm', 'ntot', 'size', 'tex', 'vlsr'])
pred_param_xgb.head()

In [ ]:
fig = plt.figure(figsize=(35, 25), dpi=400)
plt.setp(ax.get_xticklabels(), fontsize=14)
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')


markers = ['D', 's', '.']
labels = ['ntot', 'tex', 'size']
colors = ['black', 'red', 'blue']


x1 = pred_param_xgb['ntot']
y1 = pred_param_xgb['tex']
z1 = pred_param_xgb['size']
    
x2 = y_test['ntot']
y2 = y_test['tex']
z2 = y_test['size']


for i in range(len(markers)):
    ax1.scatter3D(x1, y1, z1, marker=markers[i], c=colors[i], label=labels[i])
    ax2.scatter3D(x2, y2, z2, marker=markers[i], color=colors[i], label=labels[i])

ax1.set_title('ntot-tex-size distribution - Predicted Values')
ax1.set_xlabel('Column Density')
ax1.set_ylabel('Excitation Temperature')
ax1.set_zlabel('Size')


ax2.set_title('ntot-tex-size distribution - True Values')
ax2.set_xlabel('Column Density', )
ax2.set_ylabel('Excitation Temperature')
ax2.set_zlabel('Size')
plt.legend(loc="best")

plt.savefig("5K_data/scatter3D_pred_true_5K_XGB.png")
plt.show()

### Residuals 3D plot - XGB Regressor

In [ ]:
fig = plt.figure(figsize=(15, 7), dpi=120)
plt.setp(ax.get_xticklabels(), fontsize=14)
ax1 = fig.add_subplot(121, projection='3d')

markers = ['D', 's', '.']
labels = ['ntot', 'tex', 'size']
colors = ['black', 'red', 'blue']

residuals_xgb = (y_test - y_multixgb_pred)
# residuals
x1 = residuals_xgb['ntot']
y1 = residuals_xgb['tex']
z1 = residuals_xgb['size']

for i in range(len(markers)):
    ax1.scatter3D(x1, y1, z1, marker=markers[i], c=colors[i], label=labels[i])

ax1.set_title('ntot-tex-size distribution - Residuals')
ax1.set_xlabel('Column Density')
ax1.set_ylabel('Excitation Temperature')
ax1.set_zlabel('Size')
plt.legend(loc="best")
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(45,35), dpi=250)
plt.setp(ax.get_xticklabels(), fontsize=14)
sns.set_theme(font_scale=2) 
axes[2,1].set_visible(False)
axes[2,0].set_position([0.30, 0.1,0.40,0.25])

# plot the redisual distribution using seaborn
g1 = sns.residplot(x=y_multixgb_pred[:,0], y=residuals_xgb['fwhm'], lowess=True, ax=axes[0,0]) # fwhm
g2 = sns.residplot(x=y_multixgb_pred[:,1], y=residuals_xgb['ntot'], lowess=True, ax=axes[0,1]) # column density
g3 = sns.residplot(x=y_multixgb_pred[:,2], y=residuals_xgb['size'], lowess=True, ax=axes[1,0]) # size
g4 = sns.residplot(x=y_multixgb_pred[:,3], y=residuals_xgb['size'], lowess=True, ax=axes[1,1]) # tex
g5 = sns.residplot(x=y_multixgb_pred[:,4], y=residuals_xgb['vlsr'], lowess=True, ax=axes[2,0]) # vlsr

g1.set(title='FWHM', ylabel="residuals", xlabel="predicted values")
g2.set(title='Column density', ylabel="residuals", xlabel="predicted values")
g3.set(title='Size', ylabel="predicted values", xlabel="predicted values")
g4.set(title='Tex', ylabel="residuals", xlabel="predicted values")
g5.set(title='Vlsr', ylabel="residuals", xlabel="predicted values")

plt.savefig("5K_data/residuals_XGB_5K.png")
plt.show()

## Gradient Boosting Regressor - Grid Search for Hyperparameters

In [ ]:
# model = MultiOutputRegressor(
#     GradientBoostingRegressor(
#         loss='ls', 
#         learning_rate=0.1, 
#         n_estimators=100, 
#         subsample=1.0,
#         criterion='friedman_mse', 
#         min_samples_split=2,
#         min_samples_leaf=1,
#         min_weight_fraction_leaf=0.0,
#         max_depth=3,
#         min_impurity_decrease=0.0,
#         init=None, 
#         random_state=None,
#         max_features=None,
#         alpha=0.9, 
#         verbose=0, 
#         max_leaf_nodes=None, 
#         warm_start=False,
#         validation_fraction=0.1, 
#         n_iter_no_change=None, 
#         tol=0.0001,
#         ccp_alpha=0.0))

# hyperparameters = dict(
#     estimator__learning_rate=[0.0001, 0.05, 0.1, 0.2, 0.5, 0.9], 
#     estimator__loss=['ls', 'absolute_error', 'huber'],
#     estimator__n_estimators=[10, 20, 50, 100, 200, 300, 500, 700, 1000],
#     estimator__criterion=['friedman_mse', 'squared_error'], 
#     estimator__min_samples_split=np.arange(2, 12, 2),
#     estimator__max_depth=[3, 5, 10, 15, 20, 30], 
#     estimator__min_samples_leaf=[1, 2, 3, 5, 8, 10],
#     estimator__min_impurity_decrease=[0, 0.2, 0.4, 0.6, 0.8],
#     estimator__max_leaf_nodes=[5, 10, 20, 30, 50, 100, 300])

# randomized_search = RandomizedSearchCV(
#     model, 
#     hyperparameters, 
#     random_state=0, 
#     n_iter=5, 
#     scoring=None,
#     n_jobs=2, 
#     refit=True, 
#     cv=5, 
#     verbose=True,
#     pre_dispatch='2*n_jobs', 
#     error_score='raise', 
#     return_train_score=True)

# hyperparameters_tuning = randomized_search.fit(X_train, y_train)
# print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

# tuned_model = hyperparameters_tuning.best_estimator_

In [ ]:
# print(tuned_model.predict(X_test))

In [ ]:
# # use the metrics class to calculate the metrics from the tuned model
# eval = RegressionMetrics()
# for metric in ["mae", "mse", "rmse", "mape", "r2", "msle"]:
#     print(metric, ":", eval.get_metric(metric, y_test, tuned_model.predict(X_test)))
# #
